In [1]:
import os
import requests
import zipfile
import shutil

# 로그인 위치
url = 'https://tagtog.net/-login'

# 다운로드 위치
file_url = 'https://tagtog.net/nannullna/this-is-real/-downloads/dataset-as-anndoc'
zip_file = 'download.zip'

if os.path.exists(zip_file):
    os.remove(zip_file)
    
# 로그인 정보
login_info = {
    'loginid': '', # 아이디 입력
    'password': '' # 비밀번호 입력
}

# 로그인
with requests.Session() as s:
    login_req = s.post(url, data=login_info)
    r = s.get(file_url)
    
    with open(zip_file, 'wb') as output:
        output.write(r.content)
        
# 압축 파일 풀기
folder_path = './tagtog_result'

zip_ = zipfile.ZipFile(zip_file)

if os.path.exists(folder_path):
    shutil.rmtree(folder_path)

zip_.extractall(folder_path)

os.remove(zip_file)

In [1]:
import json
import glob
import os

folder_path = './tagtog_result'

#target_folder = 'test/jeonju_hyanggyo'
target_folder = '관광지'

# 폴더 경로
root_path = os.path.join(folder_path, 'this-is-real')

json_root_path = os.path.join(root_path, 'ann.json/master/pool')
html_root_path = os.path.join(root_path, 'plain.html/pool')

json_path = os.path.join(json_root_path, target_folder)
html_path = os.path.join(html_root_path, target_folder)

def get_unique_file_name(file_name):
    return file_name[:file_name[:file_name.rfind('.')].rfind('.')]

# 파일명 목록
file_list = [get_unique_file_name(file) for file in os.listdir(html_path)]

# 파일 목록
json_file_list = os.listdir(json_path)
html_file_list = os.listdir(html_path)

files = {}
for file in file_list:
    files[file] = {'json': '', 'html': ''}

for json_file in json_file_list:
    files[get_unique_file_name(json_file)]['json'] = os.path.join(json_path, json_file)
    
for html_file in html_file_list:
    files[get_unique_file_name(html_file)]['html'] = os.path.join(html_path, html_file)

# annotation_legend
annotation_legend = os.path.join(root_path, 'annotations-legend.json')
with open(annotation_legend, 'r') as f:
    annotation_legend = json.load(f)

In [4]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

data = {
    'title': [],
    'sentence': [],
    'sentence_with_entity': [],
    'subject_entity': [],
    'object_entity': [],
    'subject_entity_word': [],
    'subject_entity_start_idx': [],
    'subject_entity_end_idx': [],
    'subject_entity_type': [],
    'object_entity_word': [],
    'object_entity_start_idx': [],
    'object_entity_end_idx': [],
    'object_entity_type': [],
    'confirm': []
}

for i, key in enumerate(files.keys()):
    # get title and sentence information from html file
    with open(files[key]['html'], 'r') as f:
        html_obj = f.read()
        
    bs_obj = BeautifulSoup(html_obj, 'html.parser')
    title, sentence = [obj.text for obj in bs_obj.select('pre')]

    data['title'].append(title)
    data['sentence'].append(sentence)


    # get entity information from json file
    entities = {
        'subj': {'word': None, 'start_idx': -1, 'end_idx': -1, 'type': None},
        'obj': {'word': None, 'start_idx': -1, 'end_idx': -1, 'type': None}
    }

    if files[key]['json'] != '':
        with open(files[key]['json'], 'r') as f:
            json_obj = json.load(f)
            
        for entity in json_obj['entities']:
            e_info, e_type = annotation_legend[entity['classId']].split('_')
            entities[e_info]['word'] = entity['offsets'][0]['text']
            entities[e_info]['start_idx'] = entity['offsets'][0]['start']
            entities[e_info]['end_idx'] = entity['offsets'][0]['start'] + len(entity['offsets'][0]['text']) - 1
            entities[e_info]['type'] = e_type

    data['subject_entity'].append(entities['subj'] if entities['subj']['word'] is not None else None)
    data['subject_entity_word'].append(entities['subj']['word'])
    data['subject_entity_start_idx'].append(entities['subj']['start_idx'])
    data['subject_entity_end_idx'].append(entities['subj']['end_idx'])
    data['subject_entity_type'].append(entities['subj']['type'])
    data['object_entity'].append(entities['obj'] if entities['obj']['word'] is not None else None)
    data['object_entity_word'].append(entities['obj']['word'])
    data['object_entity_start_idx'].append(entities['obj']['start_idx'])
    data['object_entity_end_idx'].append(entities['obj']['end_idx'])
    data['object_entity_type'].append(entities['obj']['type']) 

    # get sentence with entities information
    sentence_w_entity = sentence
    entities['subj']['symbol'] = '$$'
    entities['obj']['symbol'] = '@@'
    
    entity_list = sorted([val for val in entities.values()], key=lambda x: x['start_idx'], reverse=True)
    for entity in entity_list:
        if entity['word'] != '':
            b_str = sentence_w_entity[:entity['start_idx']]
            e_str = sentence_w_entity[entity['start_idx']:entity['end_idx']+1]
            a_str = sentence_w_entity[entity['end_idx']+1:]            
            sentence_w_entity = b_str + entity['symbol'] + e_str + entity['symbol'] + a_str    
    data['sentence_with_entity'].append(sentence_w_entity)
    
    # confirm
    data['confirm'].append(json_obj['anncomplete'])

df = pd.DataFrame(data)
df = df.sort_values('title')

In [6]:
df.head(3)

,title,sentence,sentence_with_entity,subject_entity,object_entity,subject_entity_word,subject_entity_start_idx,subject_entity_end_idx,subject_entity_type,object_entity_word,object_entity_start_idx,object_entity_end_idx,object_entity_type,confirm
1594,63빌딩,"63빌딩의 창문은 총 13,516장이 사용 되었다.","63빌딩의 창문은 총 13,516장이 사용 되었다$$$$63빌딩의 창문은 총 13,...",None,None,None,-1,-1,None,None,-1,-1,None,True
376,63빌딩,"63빌딩 내에서 사용되는 식재료는 쌀이 한달에 20,520kg으로 각 40kg당 약...","$$63빌딩$$ 내에서 사용되는 식재료는 @@쌀@@이 한달에 20,520kg으로 각...","{'word': '63빌딩', 'start_idx': 0, 'end_idx': 3,...","{'word': '쌀', 'start_idx': 19, 'end_idx': 19, ...",63빌딩,0,3,LOC,쌀,19,19,POH,True
1268,63빌딩,63빌딩 계단 오르기 대회는 1995년 개관 10주년을 맞이하여 대회가 열렸다.,$$63빌딩$$ 계단 오르기 대회는 @@1995년@@ 개관 10주년을 맞이하여 대회...,"{'word': '63빌딩', 'start_idx': 0, 'end_idx': 3,...","{'word': '1995년', 'start_idx': 16, 'end_idx': ...",63빌딩,0,3,LOC,1995년,16,20,DAT,True


In [61]:
final_df = df.copy()
final_df = final_df.loc[final_df['subject_entity'].apply(lambda x: type(x)) == dict, :]
final_df = final_df.loc[final_df['object_entity'].apply(lambda x: type(x)) == dict, :]
final_df = final_df.loc[final_df['confirm'] == True, :]
final_df = final_df.reset_index(drop=True)
final_df['index'] = [i for i in range(len(final_df))]
final_df.head()

,title,sentence,sentence_with_entity,subject_entity,object_entity,subject_entity_word,subject_entity_start_idx,subject_entity_end_idx,subject_entity_type,object_entity_word,object_entity_start_idx,object_entity_end_idx,object_entity_type,confirm,index
0,63빌딩,"63빌딩 내에서 사용되는 식재료는 쌀이 한달에 20,520kg으로 각 40kg당 약...","$$63빌딩$$ 내에서 사용되는 식재료는 @@쌀@@이 한달에 20,520kg으로 각...","{'word': '63빌딩', 'start_idx': 0, 'end_idx': 3,...","{'word': '쌀', 'start_idx': 19, 'end_idx': 19, ...",63빌딩,0,3,LOC,쌀,19,19,POH,True,0
1,63빌딩,63빌딩 계단 오르기 대회는 1995년 개관 10주년을 맞이하여 대회가 열렸다.,$$63빌딩$$ 계단 오르기 대회는 @@1995년@@ 개관 10주년을 맞이하여 대회...,"{'word': '63빌딩', 'start_idx': 0, 'end_idx': 3,...","{'word': '1995년', 'start_idx': 16, 'end_idx': ...",63빌딩,0,3,LOC,1995년,16,20,DAT,True,1
2,63빌딩,63빌딩 층수는 지하 3층 지상 60층이다. 하지만 일반인이 출입할 수 없는 위층은...,$$63빌딩$$ 층수는 지하 3층 지상 60층이다. 하지만 일반인이 출입할 수 없는...,"{'word': '63빌딩', 'start_idx': 0, 'end_idx': 3,...","{'word': '기계실(61F), 옥상(62F), 옥탑(63F, 헬기장, 안테나/...",63빌딩,0,3,LOC,"기계실(61F), 옥상(62F), 옥탑(63F, 헬기장, 안테나/첨탑)",47,85,LOC,True,2
3,63빌딩,"63빌딩의 1층부터 20층까지 운행되는 저층부 엘리베이터가 8대, 20층부터 36층...","$$63빌딩$$의 1층부터 20층까지 운행되는 저층부 엘리베이터가 8대, 20층부터...","{'word': '63빌딩', 'start_idx': 0, 'end_idx': 3,...","{'word': '1985년', 'start_idx': 185, 'end_idx':...",63빌딩,0,3,LOC,1985년,185,189,DAT,True,3
4,63빌딩,"63빌딩에는 약 34,000개의 형광등과 엘리베이터 33대, 에스컬레이터 8대, 전...","$$63빌딩$$에는 약 34,000개의 형광등과 엘리베이터 33대, 에스컬레이터 8...","{'word': '63빌딩', 'start_idx': 0, 'end_idx': 3,...","{'word': '1985년', 'start_idx': 136, 'end_idx':...",63빌딩,0,3,LOC,1985년,136,140,DAT,True,4


In [62]:
import random

final_df['sampling'] = [False for _ in range(len(final_df))]

sampling_cnt = 50

titles = random.sample(final_df['title'].unique().tolist(), sampling_cnt)

for title in titles:
    sampling_df = final_df.loc[final_df['title'] == title, :]
    sampling_index = sampling_df.sample()['index'].values[0]
    final_df.loc[final_df['index'] == sampling_index, 'sampling'] = True

len(final_df.loc[final_df['sampling'] == True, :])

50

In [63]:
final_df = final_df.drop(columns=['subject_entity', 'object_entity', 'confirm', 'index'])
final_df.head()

,title,sentence,sentence_with_entity,subject_entity_word,subject_entity_start_idx,subject_entity_end_idx,subject_entity_type,object_entity_word,object_entity_start_idx,object_entity_end_idx,object_entity_type,sampling
0,63빌딩,"63빌딩 내에서 사용되는 식재료는 쌀이 한달에 20,520kg으로 각 40kg당 약...","$$63빌딩$$ 내에서 사용되는 식재료는 @@쌀@@이 한달에 20,520kg으로 각...",63빌딩,0,3,LOC,쌀,19,19,POH,False
1,63빌딩,63빌딩 계단 오르기 대회는 1995년 개관 10주년을 맞이하여 대회가 열렸다.,$$63빌딩$$ 계단 오르기 대회는 @@1995년@@ 개관 10주년을 맞이하여 대회...,63빌딩,0,3,LOC,1995년,16,20,DAT,False
2,63빌딩,63빌딩 층수는 지하 3층 지상 60층이다. 하지만 일반인이 출입할 수 없는 위층은...,$$63빌딩$$ 층수는 지하 3층 지상 60층이다. 하지만 일반인이 출입할 수 없는...,63빌딩,0,3,LOC,"기계실(61F), 옥상(62F), 옥탑(63F, 헬기장, 안테나/첨탑)",47,85,LOC,False
3,63빌딩,"63빌딩의 1층부터 20층까지 운행되는 저층부 엘리베이터가 8대, 20층부터 36층...","$$63빌딩$$의 1층부터 20층까지 운행되는 저층부 엘리베이터가 8대, 20층부터...",63빌딩,0,3,LOC,1985년,185,189,DAT,False
4,63빌딩,"63빌딩에는 약 34,000개의 형광등과 엘리베이터 33대, 에스컬레이터 8대, 전...","$$63빌딩$$에는 약 34,000개의 형광등과 엘리베이터 33대, 에스컬레이터 8...",63빌딩,0,3,LOC,1985년,136,140,DAT,False


In [64]:
final_df.to_excel(f'관광지_relation_annotation.xlsx', index=False, encoding='utf-8')